<h1> 👋🛳️ Hello here from the Machine Learning team at <a href="https://github.com/zama-ai/concrete-ml">Zama</a></h1>

We're happy to submit our participation in this Kaggle competition. Our main idea was not only to build a predictive model that answers the question: “what sorts of people were more likely to survive?” but also to do it on encrypted data. This was possible thanks to our Python package: <a href="https://github.com/zama-ai/concrete-ml">Concrete-ML</a> that aims to simplify the use of fully homomorphic encryption (FHE) for data scientists.


# Privacy-Preserving Machine Learning on Titanic

This notebook introduces a Privacy-Preserving Machine Learning (PPML) solution to the [Kaggle Titanic competition](https://www.kaggle.com/c/titanic/) using the [Concrete-ML](https://docs.zama.ai/concrete-ml) open-source framework. Its main ambition is to show that [Fully Homomorphic Encryption](https://en.wikipedia.org/wiki/Homomorphic_encryption) (FHE) can be used for protecting data when using a Machine Learning model to predict outcomes without degrading its performance. In this example, a [XGBoost](https://xgboost.readthedocs.io/en/stable/) classifier model will be considered as it achieves near state-of-the-art accuracy.

With inspiration from the [ppxgboost repository](https://github.com/awslabs/privacy-preserving-xgboost-inference/blob/master/example/Titanic.ipynb), which is "Copyright Amazon.com, Inc. or its affiliates. All Rights Reserved. SPDX-License-Identifier: Apache-2.0".

It also took some ideas from several upvoted public notebooks, including [this one](https://www.kaggle.com/code/startupsci/titanic-data-science-solutions/notebook) from Manav Sehgal and [this one](https://www.kaggle.com/code/ldfreeman3/a-data-science-framework-to-achieve-99-accuracy#Step-3:-Prepare-Data-for-Consumption) from LD Freeman.

## Imports

In [21]:
!pip install concrete-ml

  Using cached concrete_ml-0.2.1-py3-none-any.whl (66 kB)
  Using cached concrete_numpy-0.5.0-py3-none-any.whl (92 kB)
  Using cached concrete_ml_extensions_hb-0.4.2.2-py2.py3-none-any.whl (152 kB)
  Using cached onnx-1.11.0-cp39-cp39-win_amd64.whl (11.2 MB)
  Using cached skorch-0.11.0-py3-none-any.whl (155 kB)
  Using cached onnxconverter_common-1.12.1-py2.py3-none-any.whl (83 kB)
  Using cached dill-0.3.5.1-py2.py3-none-any.whl (95 kB)
  Using cached psutil-5.9.1-cp39-cp39-win_amd64.whl (245 kB)
  Using cached onnxruntime-1.12.1-cp39-cp39-win_amd64.whl (5.8 MB)
  Using cached skl2onnx-1.12-py2.py3-none-any.whl (279 kB)
  Using cached onnxmltools-1.11.1-py3-none-any.whl (308 kB)
INFO: pip is looking at multiple versions of concrete-ml-extensions-hb[onnx] to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of protobuf to determine which version is compatible with other requirements. This could take a whil

ERROR: Could not find a version that satisfies the requirement concrete-compiler<0.5.0,>=0.4.0 (from concrete-numpy[full]) (from versions: none)
ERROR: No matching distribution found for concrete-compiler<0.5.0,>=0.4.0


In [22]:
import time
from pathlib import Path

import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV, ShuffleSplit
from tqdm import tqdm
from xgboost import XGBClassifier

from concrete.ml.sklearn import XGBClassifier as ConcreteXGBClassifier

ModuleNotFoundError: No module named 'concrete'

In [19]:
# !pip install concrete-ml

## Preprocessing the Data

Be sure to run `make download_datasets` to have local versions of the dataset.

In [3]:
# # Download the datasets
# if not Path("./local_datasets/titanic/train.csv").is_file():
#     raise ValueError("Please download the datasets by running `make download_datasets`.")

train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")
datasets = [train_data, test_data]

# Save the passenger IDs used for submission
test_ids = test_data["PassengerId"]

Let's take a closer look at the train data:

In [4]:
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


|   	| PassengerId 	| Survived 	| Pclass 	|                                              Name 	|    Sex 	|  Age 	| SibSp 	| Parch 	|           Ticket 	|    Fare 	| Cabin 	| Embarked 	|
|--:	|------------:	|---------:	|-------:	|--------------------------------------------------:	|-------:	|-----:	|------:	|------:	|-----------------:	|--------:	|------:	|---------:	|
| 0 	|           1 	|        0 	|      3 	|                           Braund, Mr. Owen Harris 	|   male 	| 22.0 	|     1 	|     0 	|        A/5 21171 	|  7.2500 	|   NaN 	|        S 	|
| 1 	|           2 	|        1 	|      1 	| Cumings, Mrs. John Bradley (Florence Briggs Th... 	| female 	| 38.0 	|     1 	|     0 	|         PC 17599 	| 71.2833 	|   C85 	|        C 	|
| 2 	|           3 	|        1 	|      3 	|                            Heikkinen, Miss. Laina 	| female 	| 26.0 	|     0 	|     0 	| STON/O2. 3101282 	|  7.9250 	|   NaN 	|        S 	|
| 3 	|           4 	|        1 	|      1 	|      Futrelle, Mrs. Jacques Heath (Lily May Peel) 	| female 	| 35.0 	|     1 	|     0 	|           113803 	| 53.1000 	|  C123 	|        S 	|
| 4 	|           5 	|        0 	|      3 	|                          Allen, Mr. William Henry 	|   male 	| 35.0 	|     0 	|     0 	|           373450 	|  8.0500 	|   NaN 	|        S 	|

We can observe:
- the target variable: Survived
- some numerical variables: PassengerID, Pclass, SbSp, Parch, Fare
- some categorical (non-numerical) variables: Name, Sex, Ticket, Cabin, Embarked

First, it seems that PassengerId and Ticket are supposed to be random Ids that should not impact the predictions so we can already remove them from the data.

In [5]:
drop_column = ["PassengerId", "Ticket"]

### Missing Values

Then, we can notice that some values are missing for the Cabin variable. We must therefore investigate a bit more about this by printing the total amounts of missing values for each variables:

In [6]:
print("-" * 3, "Train data", "-" * 3)
print(train_data.isnull().sum(), "\n")
print("-" * 3, "Test data", "-" * 3)
print(test_data.isnull().sum())

--- Train data ---
PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64 

--- Test data ---
PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64


| Training Data 	|     	|
|---------------	|-----	|
| PassengerId   	| 0   	|
| Survived      	| 0   	|
| Pclass        	| 0   	|
| Name          	| 0   	|
| Sex           	| 0   	|
| Age           	| 177 	|
| SibSp         	| 0   	|
| Parch         	| 0   	|
| Ticket        	| 0   	|
| Fare          	| 0   	|
| Cabin         	| 687 	|
| Embarked      	| 2   	|

| Test Data     	|     	|
|---------------	|-----	|
| PassengerId   	| 0   	|
| Pclass        	| 0   	|
| Name          	| 0   	|
| Sex           	| 0   	|
| Age           	| 86  	|
| SibSp         	| 0   	|
| Parch         	| 0   	|
| Ticket        	| 0   	|
| Fare          	| 1   	|
| Cabin         	| 327 	|
| Embarked      	| 0   	|

In [7]:
for incomp_var in train_data.columns:
    missing_val = pd.concat(datasets)[incomp_var].isnull().sum()
    if missing_val > 0 and incomp_var != "Survived":
        total_val = pd.concat(datasets).shape[0]
        print(f"Percentage of missing values in {incomp_var}: {missing_val/total_val*100:.1f}%")

Percentage of missing values in Age: 20.1%
Percentage of missing values in Fare: 0.1%
Percentage of missing values in Cabin: 77.5%
Percentage of missing values in Embarked: 0.2%


Percentage of missing values in Age: 20.1%

Percentage of missing values in Fare: 0.1%

Percentage of missing values in Cabin: 77.5%

Percentage of missing values in Embarked: 0.2%

Only four variables are incomplete: Cabin, Age, Embarked and Fare. However, the Cabin one seems to be missing quite more data than the others:

Since the Cabin variable misses more than 2/3 of its values, it might not be relevant to keep it:

In [8]:
drop_column += ["Cabin"]

for dataset in datasets:
    # Remove irrelevant variables
    dataset.drop(drop_column, axis=1, inplace=True)

For the other ones, we can replace the missing values using:
- the median value for Age and Fare
- the most common value for Embarked

In [9]:
for dataset in datasets:
    # Complete missing Age values with median
    dataset.Age.fillna(dataset.Age.median(), inplace=True)

    # Complete missing Embarked values with the most common one
    dataset.Embarked.fillna(dataset.Embarked.mode()[0], inplace=True)

    # Complete missing Fare values with median
    dataset.Fare.fillna(dataset.Fare.median(), inplace=True)

### Feature Engineering

We can manually extract and create new features in order to help the model interpret some behaviors and correctly predict an outcome. Those new features are:
- FamilySize: The size of the family the individual was traveling with, with 1 being someone that traveled alone. 
- IsAlone: A boolean variable stating if the individual was traveling alone (1) or not (0). This might help the model to emphasize on this idea of traveling with relatives or not.
- Title: The individual's title (Mr, Mrs, ...), often indicating a certain social status.
- Farebin and AgeBin: Binned version of the Fare and Age variables. It groups values together, generally reducing the impact of minor observation errors.

In [10]:
def get_bin_labels(bin_name, number_of_bins):
    labels = []
    for i in range(number_of_bins):
        labels.append(bin_name + f"_{i}")
    return labels


for dataset in datasets:
    # Emphasize on relatives
    dataset["FamilySize"] = dataset.SibSp + dataset.Parch + 1

    dataset["IsAlone"] = 1
    dataset.IsAlone[dataset.FamilySize > 1] = 0

    # Consider an individual's social status
    dataset["Title"] = dataset.Name.str.extract(r" ([A-Za-z]+)\.", expand=False)

    # Group fares and ages in "bins" or "buckets"
    dataset["FareBin"] = pd.qcut(dataset.Fare, 4, labels=get_bin_labels("FareBin", 4))
    dataset["AgeBin"] = pd.cut(dataset.Age.astype(int), 5, labels=get_bin_labels("AgeBin", 5))

    # Remove now-irrelevant variables
    drop_column = ["Name", "SibSp", "Parch", "Fare", "Age"]
    dataset.drop(drop_column, axis=1, inplace=True)

C:\Users\panka\AppData\Local\Temp/ipykernel_24860/3729089738.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset.IsAlone[dataset.FamilySize > 1] = 0
C:\Users\panka\AppData\Local\Temp/ipykernel_24860/3729089738.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset.IsAlone[dataset.FamilySize > 1] = 0


Let's have a look on the titles' distribution:

In [11]:
data = pd.concat(datasets)
titles = data.Title.value_counts()
print(titles)

Mr          757
Miss        260
Mrs         197
Master       61
Rev           8
Dr            8
Col           4
Mlle          2
Major         2
Ms            2
Lady          1
Sir           1
Mme           1
Don           1
Capt          1
Countess      1
Jonkheer      1
Dona          1
Name: Title, dtype: int64


| Titles  	| Counts  	|
|----------	|-----	|
| Mr       	| 757 	|
| Miss     	| 260 	|
| Mrs      	| 197 	|
| Master   	| 61  	|
| Rev      	| 8   	|
| Dr       	| 8   	|
| Col      	| 4   	|
| Mlle     	| 2   	|
| Major    	| 2   	|
| Ms       	| 2   	|
| Lady     	| 1   	|
| Sir      	| 1   	|
| Mme      	| 1   	|
| Don      	| 1   	|
| Capt     	| 1   	|
| Countess 	| 1   	|
| Jonkheer 	| 1   	|
| Dona     	| 1   	|


We can clearly observe that only a few titles represent most of the individuals. In order to prevent the model from becoming overly specific, we decide to group all the "uncommon" titles together:

In [12]:
uncommon_titles = titles[titles < 10].index

for dataset in datasets:
    dataset.Title.replace(uncommon_titles, "Rare", inplace=True)

### Dummification

Finally, we can "dummify" the remaining categorical variables. Dummification is a common technique of transforming categorical (non-numerical) data into numerical data without having to map values or consider any order between each of them. The idea is to take all the different values found in a variable and create a new associated binary variable. 

For example, the "Embarked" variable has three categorical values: "S", "C" and "Q". Dummifying the data will create three new variables, "Embarked_S", "Embarked_C" and "Embarked_Q", and set the value of "Embarked_S" (resp. "Embarked_C" and "Embarked_Q") to 1 for each data point initially labeled with "S" (resp. "C" and "Q") in the "Embarked" variable, else 0.

In [13]:
categorical_features = train_data.select_dtypes(exclude=np.number).columns
x_train = pd.get_dummies(train_data, prefix=categorical_features)
x_test = pd.get_dummies(test_data, prefix=categorical_features).to_numpy()

We then split the target variable from the others.

In [14]:
target = "Survived"
x_train = x_train.drop(columns=[target])
x_train = x_train.to_numpy()
y_train = train_data[target].to_numpy()

## Training 
### Training with XGBoost

Let's first train a classifier model using XGBoost. Since several parameters have to be fixed beforehand, we use scikit-learn's GridSearchCV method to perform cross validation in order to maximize our chance to find the best ones. 

In [15]:
# Instantiate the Cross-Validation generator
cv = ShuffleSplit(n_splits=5, test_size=0.3, random_state=0)

# Set the parameters to tune.
# Those ranges are voluntarily small in order to keep the FHE execution time per inference
# relatively low. In fact, we found out that, in this particular Titanic example, models with
# larger numbers of estimators or maximum depth don't score a much better accuracy.
param_grid = {
    "max_depth": list(range(1, 5)),
    "n_estimators": list(range(1, 5)),
    "learning_rate": [0.01, 0.1, 1],
}

# Instantiate and fit the model through grid-search cross-validation
time_begin = time.time()
model = GridSearchCV(XGBClassifier(), param_grid, cv=cv, scoring="roc_auc")
model.fit(x_train, y_train)
cv_xgb_duration = time.time() - time_begin

print(f"Best hyperparameters found in {cv_xgb_duration:.2f}s :", model.best_params_)

Best hyperparameters found in 4.84s : {'learning_rate': 1, 'max_depth': 4, 'n_estimators': 4}


Best hyperparameters found in 4.40s : {'learning_rate': 1, 'max_depth': 4, 'n_estimators': 4}


### Training with Concrete-ML

Now, let's do the same with Concrete-ML's XGBClassifier method. 

In order to do so, we need to specify the number of bits over which inputs, outputs and weights will be quantized. This value can influence the precision of the model as well as its inference running time, and therefore can lead the grid-search cross-validation to find a different set of parameters. In our case, setting this value to 2 bits outputs an excellent accuracy score while running faster. 

In [16]:
# The Concrete-ML model needs an additional parameter used for quantization
param_grid["n_bits"] = [2]

x_train = x_train.astype(np.float32)

# Instantiate and fit the model through grid-search cross-validation
time_begin = time.time()
concrete_model = GridSearchCV(ConcreteXGBClassifier(), param_grid, cv=cv, scoring="roc_auc")
concrete_model.fit(x_train, y_train)
cv_concrete_duration = time.time() - time_begin

print(f"Best hyperparameters found in {cv_concrete_duration:.2f}s :", concrete_model.best_params_)

NameError: name 'ConcreteXGBClassifier' is not defined

Best hyperparameters found in 32.79s : {'learning_rate': 0.1, 'max_depth': 4, 'n_bits': 2, 'n_estimators': 4}

## Predicting the Outcomes

Computing the predictions in FHE on the complete test set of 418 data points using the above hyperparameters can take up to 20 minutes.

In [ ]:
# Compute the predictions in clear using XGBoost
clear_predictions = model.predict(x_test)

# Compute the predictions in clear using Concrete-ML
clear_quantized_predictions = concrete_model.predict(x_test)

# Compile the Concrete-ML model on a subset
concrete_model.best_estimator_.compile(x_train[:100])

# Generate the keys
# This step is not necessary, keygen() is called directly within the predict method. However, since
# the keys are stored in cache by default, it is useful to run it beforehand in order to be able to
# measure the prediction executing time separately from the key generation one
time_begin = time.time()
concrete_model.best_estimator_.fhe_tree.keygen()
key_generation_duration = time.time() - time_begin

# Compute the predictions in FHE using Concrete-ML
# Giving x_test as an input to the predict method is possible in Concrete-ML but without the use of
# batches. We therefore decided to directly loop over it in order to better visualize the progress
# using the tqdm package, as it doesn't impact the overall execution time.
fhe_predictions = []
time_begin = time.time()
for data_point in tqdm(x_test):
    fhe_predictions.append(
        concrete_model.best_estimator_.predict(np.array([data_point]), execute_in_fhe=True)[0]
    )
prediction_duration = time.time() - time_begin

print(f"Key generation time: {key_generation_duration:.2f}s")
print(f"Total execution time for {len(clear_predictions)} inferences: {prediction_duration:.2f}s")
print(f"Execution time per inference in FHE : {prediction_duration / len(clear_predictions):.2f}s")


100%|██████████| 418/418 [18:58<00:00,  2.72s/it]

Key generation time: 12.00s

Total execution time for 418 inferences: 1138.80s

Execution time per inference in FHE : 2.72s


FHE computations are expected to be exact. This means that the model executed in FHE results in the same predictions as the Concrete-ML one, which is executed in clear and only considers quantization.

In [ ]:
number_of_equal_preds = np.sum(fhe_predictions == clear_quantized_predictions)
pred_similarity = number_of_equal_preds / len(clear_predictions) * 100
print(
    "Prediction similarity between both Concrete-ML models (quantized clear and FHE): "
    f"{pred_similarity:.2f}%",
)

Prediction similarity between both Concrete-ML models (quantized clear and FHE): 100.00%

However, as seen previously, the grid-search cross-validation was done separately between the XGBoost model and the Concrete-ML one. For this reason, the two models do not share the same set of hyperparameters, making their decision boundaries different.

Comparing how similar their predictions are one by one is thus irrelevant and only the final accuracy score given by the Kaggle platform should be considered to assess their performance.

### Save the Kaggle Submission File

When [submitted](https://www.kaggle.com/competitions/titanic/submit) to the Kaggle platform, the FHE model outputs an accuracy of 78% ([leaderboard](https://www.kaggle.com/competitions/titanic/leaderboard?search=concrete)). In comparison, the XGBoost clear one also scores around 77%.

In fact, using the given dataset, most of the submitted notebooks do not seem to exceed 79% of accuracy. Therefore, additional pre-processing and feature engineering might help increase our current score but probably not by much.

In [ ]:
# Save the FHE predictions
submission = pd.DataFrame(
    {
        "PassengerId": test_ids,
        "Survived": fhe_predictions,
    }
)
submission.to_csv("titanic_submission_fhe.csv", index=False)

# Save the XGBoost clear predictions
submission = pd.DataFrame(
    {
        "PassengerId": test_ids,
        "Survived": clear_predictions,
    }
)
submission.to_csv("titanic_submission_xgb_clear.csv", index=False)

<hr/>
If you liked this work, please have a look to our repo: **we're open source!**
<br/>
⭐️ Star us on Github here: <a href="https://github.com/zama-ai/concrete-ml" target="_blank">github.com/zama-ai/concrete-ml</a>
<br/>
👋 And ask us anything on our community forum: <a href="https://community.zama.ai/c/concrete-ml/8?utm_source=participation&utm_medium=titanic&utm_campaign=kaggle" target="_blank">community.zama.ai</a>
